# Graph Algorithms

## Graph, stack, queue & priority queue implementations

Using adjacency lists for graphs, singly-linked list for stacks, doubly-linked list for queues, and heap for priority queues. See `Graph (Adjacency Lists).ipynb`, `Linked List.ipynb` and `Heap.ipynb` for details

In [1]:
import collections

Edge = collections.namedtuple('Edge', ['from_', 'to', 'weight'])

class Graph:
    """Directed raph implemented with adjacency lists.
    
    For conciseness, edge/vertex removal.
    """
  
    def __init__(self, n=0, directed=True):
        """Constructs a graph with n nodes and 0 edges."""
        self._vertex_to_last_edge = [-1] * n # size n
        self._edge_to_destination = [] # size m
        self._edge_to_next_edge = [] # size m
        self._weights = [] # size m
        self.n = n
        self.m = 0
        self.directed = directed
        
    def display(self):
        """Prints the adjacency lists in a human readable way."""
        print()
        print('{:^8s} | {:^8s}'.format('Vertex', 'Neighbors'))
        print('---------------------------------------------')
        for v in range(self.n):
            print('{:^8d} | {}'.format(v, ', '.join(map(str, sorted(self.neighbors(v))))))
        print()
      
    def neighbors(self, i):
        """Returns the nodes directly accessible from node i."""
        self._validate(i)
        edge = self._vertex_to_last_edge[i]
        adjacent_vertices = []
        while True:
            if edge == -1:
                return sorted(adjacent_vertices)
            adjacent_vertices.append(self._edge_to_destination[edge])
            edge = self._edge_to_next_edge[edge]
            
    def add_edge(self, i, j, weight=1.0, other_side_too=None):
        """Adds an edge from node i to node j."""
        self._validate(i, j)
        if j not in self.neighbors(i):
            self._edge_to_destination.append(j)
            last_edge = self._vertex_to_last_edge[i] # can be -1
            self._edge_to_next_edge.append(last_edge)
            self._vertex_to_last_edge[i] = self.m
            self._weights.append(weight)
            self.m += 1
            if not self.directed and other_side_too is None and i != j:
                self.add_edge(j, i, weight, other_side_too=False)

    def add_vertex(self):
        """Adds a vertex."""
        self._vertex_to_last_edge.append(-1)
        self.n += 1
        
    def edges(self, v=None):
        """Returns list of edges of the graph, or of vertex v if not None."""
        if v is not None:
            self._validate(v)
            edges = []
            edge = self._vertex_to_last_edge[v]
            while edge != -1:
                to = self._edge_to_destination[edge]
                edges.append(Edge(from_=v, to=to, weight=self._weights[edge]))
                edge = self._edge_to_next_edge[edge]
            return edges
        else:
            edges = []
            for v in range(self.n):
                edge = self._vertex_to_last_edge[v]
                while edge != -1:
                    to = self._edge_to_destination[edge]
                    if self.directed or v <= to:
                        edges.append(Edge(from_=v, to=to, weight=self._weights[edge]))
                    edge = self._edge_to_next_edge[edge]
            return edges

    def _validate(self, *args):
        """Validates index."""
        for i in args:
            if i >= self.n or i < 0:
                raise ValueError('Edges must be between 0 and %d' % self.n)

In [2]:
class StackNode:
    """Node of a linear singly-linked list."""
    
    def __init__(self, value):
        self.value = value
        self.next = None


class Stack:
    """A stack."""
    
    def __init__(self):
        self.top = None
    
    def push(self, value):
        node = StackNode(value)
        node.next = self.top
        self.top = node
    
    def pop(self):
        if not self.top:
            raise ValueError('Empty stack')
        node = self.top
        self.top = self.top.next
        return node
    
    def empty(self):
        return not self.top

In [3]:
class QueueNode:
    """Node of a linear doubly-linked list."""
    
    def __init__(self, value):
        self.value = value
        self.next = None
        self.prev = None

class Queue:
    """Queue implemented as a doubly-linked list."""
    
    def __init__(self):
        self.front = None
        self.back = None
    
    def enqueue(self, value):
        """Add element to the back."""
        node = QueueNode(value)
        if not self.back:
            # Empty queue.
            self.back = node
            self.front = node
        else:
            self.back.next = node
            node.prev = self.back
            self.back = node
    
    def dequeue(self):
        """Remove element from the front."""
        if not self.back:
            raise ValueError('Empty queue')
        node = self.front
        if not self.front.next:
            # Queue is now empty.
            self.front = None
            self.back = None
        else:
            self.front.next.prev = None
            self.front = self.front.next
        return node
    
    def empty(self):
        return not self.back

In [4]:
class PriorityQueue:
    """A min-heap."""
    
    def __init__(self):
        self.array = []
        self.len = 0
        
    def items(self):
        for item in self.array:
            yield item[0]
        
    def insert(self, value, priority):
        """Inserts an element."""
        self.array.append((value, priority))
        self.len += 1
        idx = self.len - 1
        while True:
            p = self._parent(idx)
            if p is None:
                return
            if self.array[p][1] <= priority:
                return
            if self.array[p][1] > priority:
                self.array[idx], self.array[p] = self.array[p], self.array[idx]
                idx = p

    def pop(self):
        """Removes and returns min element."""
        if self.len == 0:
            raise ValueError('Empty priority queue')
        if self.len == 1:
            self.len = 0
            return self.array.pop()[0]
        top = self.array[0]
        last = self.array.pop()
        self.array[0] = last
        self.len -= 1
        idx = 0
        while True:
            left = self._left(idx)
            right = self._right(idx)
            # No child, done.
            if left is None and right is None:
                break
            # One child, necessary on the left.
            elif right is None:
                val = self.array[left][1]
                if val < last[1]:
                    self.array[idx], self.array[left] = self.array[left], self.array[idx]
                    idx = left
                    continue
                else:
                    break
            # Two children. If necessary, sift down with the larger one.
            else:
                a = self.array[left][1]
                b = self.array[right][1]
                if a < b:
                    if a < last[1]:
                        self.array[idx], self.array[left] = self.array[left], self.array[idx]
                        idx = left
                        continue
                    else:
                        break
                else:
                    if b < last[1]:
                        self.array[idx], self.array[right] = self.array[right], self.array[idx]
                        idx = right
                        continue
                    else:
                        break
        return top[0]

    def _parent(self, i):
        if i == 0:
            return None
        return (i - 1) // 2

    def _left(self, i):
        idx = 2 * i + 1
        if idx >= self.len:
            return None
        return idx

    def _right(self, i):
        idx = 2 * i + 2
        if idx >= self.len:
            return None
        return idx

## Graph Traversal

### Depth-first search (DFS)

Traverse the graph by going down a branch until a "leaf" is reached and then backtracking. Implemented using a stack.

In [5]:
def depth_first_search(g, func, start_vertex=0):
    """Pre-order depth-first search."""
    s = Stack()
    s.push(start_vertex)
    seen = {start_vertex}
    while not s.empty():
        v = s.pop().value
        func(v) # pre-order
        for n in g.neighbors(v):
            if n not in seen:
                s.push(n)
                seen.add(n)

In [6]:
g = Graph(10)
g.add_edge(0, 2)
g.add_edge(2, 3)
g.add_edge(2, 7)
g.add_edge(2, 8)
g.add_edge(4, 4)
g.add_edge(4, 5)
g.add_edge(5, 6)
g.add_edge(7, 2)
g.add_edge(7, 3)
g.add_edge(7, 4)
g.add_edge(7, 5)
g.add_edge(7, 6)
g.add_edge(8, 7)
g.add_edge(8, 9)
g.add_edge(9, 1)
g.display()

depth_first_search(g, lambda v: print('Visiting %d' % v))


 Vertex  | Neighbors
---------------------------------------------
   0     | 2
   1     | 
   2     | 3, 7, 8
   3     | 
   4     | 4, 5
   5     | 6
   6     | 
   7     | 2, 3, 4, 5, 6
   8     | 7, 9
   9     | 1

Visiting 0
Visiting 2
Visiting 8
Visiting 9
Visiting 1
Visiting 7
Visiting 6
Visiting 5
Visiting 4
Visiting 3


![Graph](img/graph_bfs_dfs.png)

### Breadth-first search (BFS)

Traverse the graph "level by level." Implemented using a queue.

In [7]:
def breadth_first_search(g, func, start_vertex=0):
    """Pre-order breadth-first search."""
    q = Queue()
    q.enqueue(start_vertex)
    seen = {start_vertex}
    while not q.empty():
        v = q.dequeue().value
        func(v) # pre-order
        for n in g.neighbors(v):
            if n not in seen:
                q.enqueue(n)
                seen.add(n)

In [8]:
g.display()

breadth_first_search(g, lambda v: print('Visiting %d' % v))


 Vertex  | Neighbors
---------------------------------------------
   0     | 2
   1     | 
   2     | 3, 7, 8
   3     | 
   4     | 4, 5
   5     | 6
   6     | 
   7     | 2, 3, 4, 5, 6
   8     | 7, 9
   9     | 1

Visiting 0
Visiting 2
Visiting 3
Visiting 7
Visiting 8
Visiting 4
Visiting 5
Visiting 6
Visiting 9
Visiting 1


## Topological Sort (for DAGs)

Directed Acyclic Graphs are equivalent to graphs that have a topological sort, i.e. partial order. The goal is to list all vertices so that if there's an edge from u to v, u comes before v in the list.

The naïve way: take any node with no incoming edge. Remove outgoing edges. Repeat. It works but depending on the complexity of removing edges, it can be slow. Instead we can use a non-destructive algorithms that relies on queues.

Precompute deg(v) for each node v, which is the number of incoming edge to v. Put all the ones with dev(v) = 0 in a queue. Then take the front of the queue as the next element, decrease deg for all its neighbors, and if one reaches 0 then add it to the queue. Repeat until the queue is empty.

In [9]:
def topological_sort(g):
    """Topological sort for a DAG."""
    deg = [0] * g.n # contains number of incoming edges
    for v in range(g.n):
        for n in g.neighbors(v):
            deg[n] += 1
    
    sorted_vertices = []
    
    q = Queue() # contains vertices with deg(v) = 0
    for v in range(g.n):
        if deg[v] == 0:
            q.enqueue(v)
    
    while not q.empty():
        v = q.dequeue().value
        sorted_vertices.append(v)
        for n in g.neighbors(v):
            deg[n] -= 1
            if deg[n] == 0:
                q.enqueue(n)
    
    return sorted_vertices

In [10]:
g = Graph(9)
g.add_edge(6, 2)
g.add_edge(2, 5)
g.add_edge(2, 8)
g.add_edge(1, 2)
g.add_edge(2, 7)
g.add_edge(1, 4)
g.add_edge(4, 8)
g.add_edge(3, 4)
g.add_edge(3, 7)

g.display()


 Vertex  | Neighbors
---------------------------------------------
   0     | 
   1     | 2, 4
   2     | 5, 7, 8
   3     | 4, 7
   4     | 8
   5     | 
   6     | 2
   7     | 
   8     | 



![Graph](img/graph_topological_sort.png)

In [11]:
print(topological_sort(g))

[0, 1, 3, 6, 4, 2, 5, 7, 8]


The order is better seen on this graph:

![Graph](img/graph_topological_sort_2.png)

## Minimum Spanning Tree (MST)

Goal: for G = (V, E) undirected, construct a tree using a subset of E that connects all vertices.

### Kruskal's algorithm

Idea 1: Edge with minimum weight must be included.

Idea 2: Once an edge is included, we merge the connected parts into supernodes, using Union-Find.

Union-Find uses a array of size |v|, initialized to A(v) = v, and supports two operations:
  * **Find:** two vertices are connected if A(u) = A(v)
  * **Union:** merge two groups by settings every A(u') where A(u') = A(u) to A(v)

In [12]:
def kruskal(g):
    edges = sorted(g.edges(), key=lambda e: e.weight)
    selected_edges = []
    a = list(range(g.n)) # A(v) = v
    for e in edges:
        if a[e.from_] != a[e.to]: # "find"
            # Not yet connected, let's do it now.
            selected_edges.append(e)
            old, new = a[e.to], a[e.from_]
            for i in range(g.n):
                if a[i] == old:
                    a[i] = new # "union"
        # Else, ignore the edge.
    k = Graph(g.n, directed=False)
    cost = 0.0
    for e in selected_edges:
        k.add_edge(e.from_, e.to, e.weight)
        cost += e.weight
    return k, cost

In [13]:
g = Graph(9, directed=False)

# Use weight = from + to for simplicity.
g.add_edge(0, 6, 6.0)
g.add_edge(1, 6, 7.0)
g.add_edge(1, 7, 8.0)
g.add_edge(2, 5, 7.0)
g.add_edge(2, 7, 9.0)
g.add_edge(2, 8, 10.0)
g.add_edge(3, 4, 7.0)
g.add_edge(3, 6, 9.0)
g.add_edge(3, 7, 10.0)
g.add_edge(4, 5, 9.0)
g.add_edge(5, 8, 13.0)
g.add_edge(7, 8, 15.0)
g.display()


 Vertex  | Neighbors
---------------------------------------------
   0     | 6
   1     | 6, 7
   2     | 5, 7, 8
   3     | 4, 6, 7
   4     | 3, 5
   5     | 2, 4, 8
   6     | 0, 1, 3
   7     | 1, 2, 3, 8
   8     | 2, 5, 7



![Graph](img/graph_mst.png)

In [14]:
k, cost = kruskal(g)
k.display()
print('Cost: %r' % cost)


 Vertex  | Neighbors
---------------------------------------------
   0     | 6
   1     | 6, 7
   2     | 5, 7, 8
   3     | 4, 6
   4     | 3
   5     | 2
   6     | 0, 1, 3
   7     | 1, 2
   8     | 2

Cost: 63.0


![Graph](img/graph_mst_solved.png)

### Prim's algorithm

* Pick any node and put it in S.
* Find edge e with smallest weight that connects a vertex in S to a vertex v not yet in S.
* Add e to MST and v to S.
* Repeat n times (until S = V).

In [15]:
def prims(g):
    u = 0 # start with node 0
    S = {u}
    edges = PriorityQueue() # edges connecting S and its complement
    selected_edges = []
    while len(S) != g.n:
        for e in g.edges(u):
            if e.to not in S:
                edges.insert(e, e.weight)
        while True:
            e = edges.pop()
            if e.to not in S:
                selected_edges.append(e)
                S.add(e.to)
                u = e.to
                break
    p = Graph(g.n, directed=False)
    cost = 0.0
    for e in selected_edges:
        p.add_edge(e.from_, e.to, e.weight)
        cost += e.weight
    return p, cost

In [16]:
p, cost = prims(g)
p.display()
print('Cost: %r' % cost)


 Vertex  | Neighbors
---------------------------------------------
   0     | 6
   1     | 6, 7
   2     | 5, 7, 8
   3     | 4, 6
   4     | 3
   5     | 2
   6     | 0, 1, 3
   7     | 1, 2
   8     | 2

Cost: 63.0
